In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys 
sys.path.append('../')

In [4]:
import os
import tensorflow 
import numpy as np
import random


seed_value = 42
#seed_value = None

environment_name = sys.executable.split('/')[-3]
print('Environment:', environment_name)
os.environ[environment_name] = str(seed_value)

np.random.seed(seed_value)
random.seed(seed_value)
tensorflow.random.set_seed(seed_value)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow.compat.v1.keras.backend as K
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
K.set_session(session)

tensorflow.__version__

Environment: biotmpygpu


'2.2.0'

In [5]:
#multiple_gpus=[0,1,2,3]
multiple_gpus=None

In [7]:
import os
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

if multiple_gpus:
    devices = []
    for gpu in multiple_gpus:
        devices.append('/gpu:' + str(gpu))    
    strategy = tensorflow.distribute.MirroredStrategy(devices=devices)
# Get the GPU device name.
else:
    device_name = tensorflow.test.gpu_device_name()
    # The device name should look like the following:
    if device_name == '/device:GPU:0':
        print('Found GPU at: {}'.format(device_name))
    else:
        raise SystemError('GPU device not found')

    os.environ["CUDA_VISIBLE_DEVICES"] = device_name
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    
    
from wrappers.bioc_wrapper import bioc_to_docs, bioc_to_relevances
from wrappers.pandas_wrapper import relevances_to_pandas, docs_to_pandasdocs
from preprocessing.dl import DL_preprocessing
from mlearning.dl_models import Burns_CNN, Burns_LSTM, Chollet_DNN, DNN, Burns_CNN2, Burns_BiLSTM, Burns_CNN3, Burns_CNNBiLSTM
from mlearning.dl_models import DeepDTA
from preprocessing.embeddings import compute_embedding_matrix, glove_embeddings_2
from preprocessing.ml import serialize_config, deserialize_config
from preprocessing.dl import average_precision
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from preprocessing.config import DLConfig
from tensorflow.keras.preprocessing import text
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
import seaborn as sns
from openpyxl import load_workbook
import pandas as pd
import os
from keras import backend as K
import pickle
#from tfdeterminism import patch
#patch()

Num GPUs Available:  4
Found GPU at: /device:GPU:0


[nltk_data] Downloading package stopwords to /home/malves/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/malves/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/malves/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
config = DLConfig()
config = config.load('models/biobert_lstm_ft/biobert_lstm_ft_4/config.txt')
model = config.load_tensorflow_model('models/biobert_lstm_ft/biobert_lstm_ft_4/model_tf')
config.model_id

TypeError: The two structures don't have the same nested structure.

First structure: type=dict str={'input_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name='inputs/input_ids')}

Second structure: type=list str=[TensorSpec(shape=(None, 512), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, 512), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, 512), dtype=tf.int32, name='inputs/2')]

More specifically: The two namedtuples don't have the same sequence type. First structure type=dict str={'input_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name='inputs/input_ids')} has type dict, while second structure type=list str=[TensorSpec(shape=(None, 512), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, 512), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, 512), dtype=tf.int32, name='inputs/2')] has type list
Entire first structure:
{'input_ids': .}
Entire second structure:
[., ., .]

In [21]:
test_dataset_path = '../datasets/PMtask_Triage_TestSet.xml'

docs_test = bioc_to_docs(test_dataset_path, config=config)
relevances_test = bioc_to_relevances(test_dataset_path, 'protein-protein')

x_test_df = docs_to_pandasdocs(docs_test)
y_test_df = relevances_to_pandas(x_test_df, relevances_test)

x_test, y_test = DL_preprocessing(x_test_df, y_test_df, config, dataset = 'test')

nmr_unknown_words=0
total_words = 0
for doc in x_test:
    for sentence in doc:
        for word in sentence:
            if word != 0:
                total_words += 1
            if word == 1:
                nmr_unknown_words+=1

print('Percentage of Unknown Words on the Test Set: {:.2%} ({} in {})'.format(nmr_unknown_words/total_words, nmr_unknown_words,total_words))

Index of Unknown Words: 1
Percentage of Unknown Words on the Test Set: 15.81% (42749 in 270451)


In [22]:
x_test_df['Document'][0].fulltext_string

'high - conductance calcium - activated potassium channels rat brain pharmacology distribution subunit composition rat brain high - conductance ca2+ - activated k+ bk channels targeted axons nerve terminals knaus h. g. et al 1996 j. neurosci 16 955 - 963 absolute levels regional expression subunit composition yet fully established investigate issues ibtx analogue 125i ibtx - d19y/y36f employed selectively binds neuronal bk channels high affinity kd 21 pm cross - linking experiments 125i ibtx - d19y/y36f presence bifunctional reagent led covalent incorporation radioactivity protein apparent molecular mass 25 kda deglycosylation immunoprecipitation studies antibodies raised alpha - smooth muscle beta - subunits bk channel suggest beta - subunit associated neuronal bk channel novel protein quantitative receptor autoradiography reveals highest levels bk channel expression outer layers neocortex hippocampal perforant path projections interpeduncular nucleus distribution pattern also confirm

In [24]:
yhat_probs = model.predict(x_test, verbose=0)
yhat_probs = yhat_probs[:, 0]

yhat_classes = np.where(yhat_probs > 0.5, 1, yhat_probs)
yhat_classes = np.where(yhat_classes < 0.5, 0, yhat_classes).astype(np.int64)

In [11]:
# accuracy: (tp + tn) / (p + n)
avg_prec  = average_precision(y_test_df, yhat_probs)
print('Average Precision: %f' % avg_prec)

accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

# kappa
kappa = cohen_kappa_score(y_test, yhat_classes)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(y_test, yhat_probs)
print('ROC AUC: %f' % auc)

mcc = matthews_corrcoef(y_test, yhat_classes)
print('MCC: %f' % mcc)

# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)


tn, fp, fn, tp = confusion_matrix(y_test, yhat_classes).ravel()

Average Precision: 0.732853
Accuracy: 0.658725
Precision: 0.624856
Recall: 0.771307
F1 score: 0.690401
Cohens kappa: 0.319424
ROC AUC: 0.736619
MCC: 0.328270
[[397 326]
 [161 543]]


In [12]:
yhat_probs_copy = yhat_probs.copy()
yhat_probs_df = pd.Series(data=yhat_probs_copy.reshape(1427,), index=y_test_df.index)
yhat_probs_df[yhat_probs_df < 0.50] = - (1 - yhat_probs_df)
sorted_yhat = yhat_probs_df.sort_values(ascending=False)
ranked_df = sorted_yhat.to_frame().rename(columns={0: 'Conf. Score'})
data = {'PMID':[],'URL':[]}
for index in ranked_df.index.values:
    doc_title = x_test_df['Document'][index].raw_title
    data['PMID'].append(index)
    data['URL'].append('<a href="https://pubmed.ncbi.nlm.nih.gov/{0}/">{1}</a>'.format(index, doc_title))
meta_df = pd.DataFrame(data, index= ranked_df.index)

final_df = meta_df.join(ranked_df)
final_df
import pickle
with open('final_df.txt', 'wb') as f:
    pickle.dump(final_df, f)

In [14]:
from IPython.core.display import display, HTML
display(HTML(final_df.to_html(escape=False, index=False)))

PMID,URL,Conf. Score
9548595,Identification of the epitope of a monoclonal antibody that inhibits heparin binding of lipoprotein lipase: new evidence for a carboxyl-terminal heparin-binding domain.,0.974700
8662907,Interaction between the amino-terminal SH3 domain of CRK and its natural target proteins.,0.974377
17689488,A Sveinsson's chorioretinal atrophy-associated missense mutation in mouse Tead1 affects its interaction with the co-factors YAP and TAZ.,0.973480
10627461,Cytoplasmic domains of GpIbalpha and GpIbbeta regulate 14-3-3zeta binding to GpIb/IX/V.,0.969070
7721887,Identification of three tyrosine residues of glycoprotein Ib alpha with distinct roles in von Willebrand factor and alpha-thrombin binding.,0.968821
10713702,Dimerization of the amino terminal domain of p57Kip2 inhibits cyclin D1-cdk4 kinase activity.,0.967892
9751730,Vascular endothelial growth factor B (VEGF-B) binds to VEGF receptor-1 and regulates plasminogen activator activity in endothelial cells.,0.964524
10413511,An aspartate residue at the extracellular boundary of TMII and an arginine residue in TMVII of the gastrin-releasing peptide receptor interact to facilitate heterotrimeric G protein coupling.,0.964389
10037723,The interaction of the human MutL homologues in hereditary nonpolyposis colon cancer.,0.963925
12021405,Molecular basis of the selectivity of gastrin-releasing peptide receptor for gastrin-releasing peptide.,0.960623
